In [150]:
import pandas as pd
import numpy as np
from snappy import *
#from sage.all import *


In [ ]:
#load data

#knotinfo = pd.read_excel('/Users/seohyeonlee/knotinfo/data/knotinfo_data_complete.xls')
linkinfo = pd.read_excel('/Users/seohyeonlee/knotinfo/data/linkinfo_data_complete.xls')


In [ ]:
linkinfo_glossary = linkinfo.iloc[0, :].copy()
linkinfo_glossary['determinant']

linkinfo_glossary.shape


(89,)

In [122]:
linkinfo.shape
linkinfo.describe()
linkinfo.isna().any()
linkinfo.head()

#remove first row
linkinfo = linkinfo.iloc[1:, :].reset_index(drop=True)
linkinfo


,name,name_anon,diagram,diagram_anon,name_unoriented,name_unoriented_anon,category,category_anon,knot_atlas,knot_atlas_anon,...,determinant_anon,signature,signature_anon,nullity,nullity_anon,unlinking_number,unlinking_number_anon,Unnamed: 86,weak_splitting_number,weak_splitting_number_anon
0,L2a1{0},diagram_display.php?L2a1{0},linkL2a1{0}-50.png,diagram_display.php?L2a1{0},L2a1,NaN,2,NaN,L2a1,http://katlas.math.toronto.edu/wiki/L2a1,...,NaN,1,NaN,0,NaN,1,NaN,NaN,1,NaN
1,L2a1{1},diagram_display.php?L2a1{1},linkL2a1{1}-50.png,diagram_display.php?L2a1{1},L2a1,NaN,2,NaN,L2a1,http://katlas.math.toronto.edu/wiki/L2a1,...,NaN,-1,NaN,0,NaN,1,NaN,NaN,1,NaN
2,L4a1{0},diagram_display.php?L4a1{0},linkL4a1{0}-50.png,diagram_display.php?L4a1{0},L4a1,NaN,4,NaN,L4a1,http://katlas.math.toronto.edu/wiki/L4a1,...,NaN,1,NaN,0,NaN,2,NaN,NaN,2,NaN
3,L4a1{1},diagram_display.php?L4a1{1},linkL4a1{1}-50.png,diagram_display.php?L4a1{1},L4a1,NaN,4,NaN,L4a1,http://katlas.math.toronto.edu/wiki/L4a1,...,NaN,-3,NaN,0,NaN,2,NaN,NaN,2,NaN
4,L5a1{0},diagram_display.php?L5a1{0},linkL5a1{0}-50.png,diagram_display.php?L5a1{0},L5a1,NaN,5,NaN,L5a1,http://katlas.math.toronto.edu/wiki/L5a1,...,NaN,1,NaN,0,NaN,1,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#4000+ knots missing name
linkinfo['name'].value_counts()

type(linkinfo['name'][8373])

noname = linkinfo.loc[linkinfo['name'].isna()]
noname.isna().sum()


name                          4187
name_anon                     4187
diagram                       4187
diagram_anon                  4187
name_unoriented               4187
                              ... 
unlinking_number              4187
unlinking_number_anon         4187
Unnamed: 86                   4187
weak_splitting_number         4187
weak_splitting_number_anon    4187
Length: 89, dtype: int64

In [ ]:
#remove rows where all values are NaN. do the same with columns
linkinfo = linkinfo.dropna(axis=0, how='all')
linkinfo = linkinfo.dropna(axis=1, how='all')


In [ ]:
#sage

#extract properties needed to determine if link is ribbon
alexander = linkinfo['multivariable_alexander']
jones = linkinfo['jones_polynomial']
linking_matrix = linkinfo['linking_matrix']
#check jones polynomial
#check if factorable by the jones polynomial of the unlink with same components: (q + q^{-1})^{n-1}
linkinfo['jones_polynomial']


0                                          -x^(-5)-x^(-1)
1                                                  -x-x^5
2                          -x^(-9)-x^(-5) + x^(-3)-x^(-1)
3                                     -x^3-x^7 + x^9-x^11
4                       x^(-7)-2/x^5 + x^(-3)-2/x + x-x^3
                              ...                        
4183    -x^(-5) + x^(-3)-3/x-2*x^3-x^5-2*x^9 + x^11-2*...
4184    -x^(-17) + x^(-15)-3/x^13-2/x^9-x^(-7)-2/x^3 +...
4185    -x^(-11) + x^(-9)-3/x^7-2/x^3-x^(-1)-2*x^3 + x...
4186    -x^(-11) + x^(-9)-3/x^7-2/x^3-x^(-1)-2*x^3 + x...
4187    -x^7 + x^9-3*x^11-2*x^15-x^17-2*x^21 + x^23-2*...
Name: jones_polynomial, Length: 4188, dtype: object

In [126]:
import  re

#process linking matrices
#'matrix_zero' is true if the linking matrix consists of only 0s, false if not, NA if data is missing

def parse_matrix(row):
	if not pd.isna(row):
		nums = re.findall(r'-?\d+', row)
		#print(nums)
		#print(all(num == 0 for num in nums))
		all_zero = all(int(num) == 0 for num in nums)
		#row = row.replace('{', '[').replace('}', ']')
	else:
		all_zero = pd.NA
	return all_zero

linkinfo['matrix_zero'] = linking_matrix.apply(lambda x: parse_matrix(x))


In [ ]:
matrix_zero_df = linkinfo[['name_unoriented', 'matrix_zero']].drop_duplicates(keep='first')
#matrix_zero_df = matrix_zero_df.set_index(matrix_zero_df['name_unoriented'], drop=True)

matrix_zero_df.reset_index(drop=True)


matrix_zero_df.to_csv('results/matrix_zero.csv')
matrix_zero_df.to_excel('results/matrix_zero.xlsx')


In [ ]:
"""
nonzero = linkinfo.loc[linkinfo['matrix_zero'] == False][['name_unoriented']].values.tolist()
zero = linkinfo.loc[linkinfo['matrix_zero']][['name_unoriented']].values.tolist()

links_with_nonzero_matrices = []
for item in nonzero:
	name = item[0]
	if name not in set(links_with_nonzero_matrices):
		links_with_nonzero_matrices.append(name)

links_with_zero_matrices = []
for item in zero:
	name = item[0]
	if name not in set(links_with_zero_matrices):
		links_with_zero_matrices.append(name)

links_with_nonzero_matrices = pd.Series(links_with_nonzero_matrices)
links_with_zero_matrices = pd.Series(links_with_zero_matrices)

#export list of links with nonzero linking matrices

links_with_nonzero_matrices.to_csv('results/nonzero_linking_matrix.csv')
links_with_nonzero_matrices.to_excel('results/nonzero_linking_matrix.xlsx')
"""


In [ ]:
determinant_is_zero = linkinfo['determinant'].apply(lambda row: row == 0)

#158 links whose determinant is zero
linkinfo['determinant_is_zero'] = determinant_is_zero
linkinfo['determinant_is_zero']


0       False
1       False
2       False
3       False
4       False
        ...  
4183     True
4184     True
4185     True
4186     True
4187     True
Name: determinant_is_zero, Length: 4188, dtype: bool

In [ ]:
import math
#test if the determinant is a perfect square
determinant_is_square = linkinfo['determinant'].apply(lambda row: math.sqrt(row) % 1 == 0)
determinant_is_square

linkinfo['determinant_is_square'] = determinant_is_square


In [229]:

#aggregate information needed to check whether link is ribbon (add more properties later)
ribbon_info = linkinfo[['name_unoriented', 'linking_matrix', 'determinant', 'matrix_zero', 'determinant_is_zero']]
ribbon_info = ribbon_info.drop_duplicates(keep='first')

matrix_and_determinant_both_zero = ribbon_info['matrix_zero'] * ribbon_info['determinant_is_zero']

ribbon_info['matrix_and_determinant_both_zero'] = matrix_and_determinant_both_zero

matrix_and_determinant_both_zero.value_counts()

ribbon_info[matrix_and_determinant_both_zero].name_unoriented.tolist()


['L9n27',
 'L10n32',
 'L10n36',
 'L10n56',
 'L10n57',
 'L10n59',
 'L10n107',
 'L11n218',
 'L11n226',
 'L11n244',
 'L11n247',
 'L11n381',
 'L11n396',
 'L11n404',
 'L11n406',
 'L11n436']

In [ ]:
ribbon_info = ribbon_info.reset_index(drop=True)
ribbon_info


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,matrix_and_determinant_both_zero
0,L2a1,"{{0, -1}, {-1, 0}}",2,False,False,False
1,L2a1,"{{0, 1}, {1, 0}}",2,False,False,False
2,L4a1,"{{0, -2}, {-2, 0}}",4,False,False,False
3,L4a1,"{{0, 2}, {2, 0}}",4,False,False,False
4,L5a1,"{{0, 0}, {0, 0}}",8,True,False,False
...,...,...,...,...,...,...
3511,L11n459,"{{0, 1, 1, 0}, {1, 0, 1, -1}, {1, 1, 0, -1}, {...",0,False,True,False
3512,L11n459,"{{0, -1, -1, 0}, {-1, 0, 1, -1}, {-1, 1, 0, -1...",0,False,True,False
3513,L11n459,"{{0, 1, -1, 0}, {1, 0, -1, 1}, {-1, -1, 0, -1}...",0,False,True,False
3514,L11n459,"{{0, -1, 1, 0}, {-1, 0, -1, -1}, {1, -1, 0, 1}...",0,False,True,False


In [ ]:
#knots whose linking matrix are zero
ribbon_info[ribbon_info['matrix_zero']]


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,matrix_and_determinant_both_zero
4,L5a1,"{{0, 0}, {0, 0}}",8,True,False,False
11,L6a4,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,False
20,L7a1,"{{0, 0}, {0, 0}}",24,True,False,False
23,L7a3,"{{0, 0}, {0, 0}}",16,True,False,False
24,L7a4,"{{0, 0}, {0, 0}}",16,True,False,False
...,...,...,...,...,...,...
3261,L11n404,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,True
3264,L11n406,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,True
3267,L11n408,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,False
3268,L11n409,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",80,True,False,False


In [ ]:
#knots whose determinant (calculated by plugging -1 into the alexander polynomial) are zero
ribbon_info[ribbon_info['determinant_is_zero']]


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,matrix_and_determinant_both_zero
104,L8n6,"{{0, -2, 0}, {-2, 0, -2}, {0, -2, 0}}",0,False,True,False
105,L8n6,"{{0, 2, 0}, {2, 0, 2}, {0, 2, 0}}",0,False,True,False
106,L8n6,"{{0, -2, 0}, {-2, 0, 2}, {0, 2, 0}}",0,False,True,False
107,L8n6,"{{0, 2, 0}, {2, 0, -2}, {0, -2, 0}}",0,False,True,False
116,L8n8,"{{0, -1, 1, 0}, {-1, 0, 0, 1}, {1, 0, 0, -1}, ...",0,False,True,False
...,...,...,...,...,...,...
3511,L11n459,"{{0, 1, 1, 0}, {1, 0, 1, -1}, {1, 1, 0, -1}, {...",0,False,True,False
3512,L11n459,"{{0, -1, -1, 0}, {-1, 0, 1, -1}, {-1, 1, 0, -1...",0,False,True,False
3513,L11n459,"{{0, 1, -1, 0}, {1, 0, -1, 1}, {-1, -1, 0, -1}...",0,False,True,False
3514,L11n459,"{{0, -1, 1, 0}, {-1, 0, -1, -1}, {1, -1, 0, 1}...",0,False,True,False


In [ ]:
ribbon_info[ribbon_info['matrix_zero'] & ribbon_info['matrix_zero']]


,name_unoriented,linking_matrix,determinant,matrix_zero,determinant_is_zero,matrix_and_determinant_both_zero
4,L5a1,"{{0, 0}, {0, 0}}",8,True,False,False
11,L6a4,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,False
20,L7a1,"{{0, 0}, {0, 0}}",24,True,False,False
23,L7a3,"{{0, 0}, {0, 0}}",16,True,False,False
24,L7a4,"{{0, 0}, {0, 0}}",16,True,False,False
...,...,...,...,...,...,...
3261,L11n404,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,True
3264,L11n406,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",0,True,True,True
3267,L11n408,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",16,True,False,False
3268,L11n409,"{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}",80,True,False,False


In [ ]:
#export the version of the linkinfo table with empty values removed and links with nonzero linking matrices identified in a separate column
ribbon_info.to_csv('results/linkinfo_with_ribbon_info.csv', index=False)
ribbon_info.to_excel('results/linkinfo_with_ribbon_info.xlsx', index=False)
